In [1]:
__SCRIPT_VERSION__ = "3.5"
__SCRIPT_DATE__ = "26/May/2023"

In [2]:
from fastlane_bot.bot import CarbonBot as Bot
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, Pair
from fastlane_bot.tools.analyzer import CPCAnalyzer
from fastlane_bot.tools.cryptocompare import CryptoCompare
import requests
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCAnalyzer))
import pandas as pd
import datetime
import time
import json
from hashlib import md5
from fastlane_bot import __VERSION__

Using default database url, if you want to use a different database, set the backend_url found at the bottom of manager_base.py
CarbonBot v3-b2.1 (03/May/2023)
ConstantProductCurve v2.14 (23/May/2023)
CPCAnalyzer v1.5 (18/May/2023)


# Mainnet Arbitrage Monitoring Bot [A015 - v3.5SKL]
_v3.5 SKL; contains changes on notifications and excluded curves_

In [3]:
cid = lambda pair: md5(pair.encode()).hexdigest()
cid("WETH-6Cc2/USDC-eB48")

'ead90114986e463b0157c49422d8d465'

In [4]:
bot     = Bot()
CCm     = bot.get_curves()
fn = f"../data/A014-{int(time.time())}.csv.gz"
print (f"Saving curve data as {fn}")
CCm.asdf().to_csv(fn, compression = "gzip")

Saving curve data as ../data/A014-1685089861.csv.gz


In [5]:
class TokenAddress():
    def __init__(self, db):
        self._db = db
        
    def addr_from_ticker(self, ticker):
        return self._db.get_token(key=ticker).address
    a = addr_from_ticker
    
    def ticker_from_addr(self, addr):
        raise NotImplemented()
TA = TokenAddress(bot.db)      
TA.a("WETH-6Cc2")

'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

#### Examining specific curves

In [6]:
c = CCm.bycid0("7382-1")[0]
c

ConstantProductCurve(k=3.845403938030023e+16, x=8434451122.601829, x_act=0, y_act=0.4559162766718673, pair='USDC-eB48/WETH-6Cc2', cid='1701411834604692317316873037158841057382-1', fee=0.002, descr='NaN', constr='carb', params={'exchange': 'carbon_v1', 'tknx_dec': 18, 'tkny_dec': 6, 'tknx_addr': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'tkny_addr': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', 'blocklud': 17342093, 'y': 0.4559162766718673, 'yint': 0.4559162766718673, 'A': 0, 'B': 0.023249526586287494, 'pa': 0.0005405405405405376, 'pb': 0.0005405405405405376})

In [7]:
c.p_min, c.p_max, c.p, 1/c.p_min, 1/c.p_max, 1/c.p

(0.000540540486486489,
 0.0005405405945945916,
 0.0005405405945945916,
 1850.00018500001,
 1849.9998150000288,
 1849.9998150000288)

In [8]:
cp = c.params
cp.pa, cp.pb, 1/cp.pa, 1/cp.pb

(0.0005405405405405376,
 0.0005405405405405376,
 1850.00000000001,
 1850.00000000001)

In [9]:
print(c.description())

cid      = 057382-1 [1701411834604692317316873037158841057382-1]
primary  = WETH/USDC [WETH-6Cc2/USDC-eB48]
pp       = 1,849.999815 USDC per WETH
pair     = USDC/WETH [USDC-eB48/WETH-6Cc2]
tknx     =             0.000000 USDC-eB48  [virtual:    8,434,451,122.602]
tkny     =             0.455916 WETH-6Cc2  [virtual:        4,559,163.225]
p        = 0.0005405405945945916 [min=0.000540540486486489, max=0.0005405405945945916] WETH-6Cc2 per USDC-eB48
fee      = 0.002
descr    = NaN



## Header and metadata

In [10]:
now = datetime.datetime.now()
print("\n\n")
print("*"*100)
print("*"*100)
print(f"ARBITRAGE ANALYSIS RUN @ {now.isoformat().split('.')[0]}Z [{int(now.timestamp())}]")
print("*"*100)
print("*"*100)




****************************************************************************************************
****************************************************************************************************
ARBITRAGE ANALYSIS RUN @ 2023-05-26T09:31:01Z [1685089861]
****************************************************************************************************
****************************************************************************************************


## Read curves

### Read Carbon curves

In [11]:
#CCm     = CPCContainer.from_df(pd.read_csv("../data/A014-1684148163.csv.gz"))
CCc1_noexcl  = CCm.byparams(exchange="carbon_v1")              # all Carbon positions
CCnc1        = CCm.byparams(exchange="carbon_v1", _inv=True)   # all non-Carbon positions

#### Remove curves

In [12]:
c = CCc1_noexcl.bycid0("749015-0")[0]
1/c.p_min, 1/c.p_max
c

ConstantProductCurve(k=1343185108526.0225, x=1090256.5698385532, x_act=0, y_act=50079.3888655038, pair='BNT-FF1C/vBNT-7f94', cid='4423670769972200025023869896612986749015-0', fee=0.002, descr='NaN', constr='carb', params={'exchange': 'carbon_v1', 'tknx_dec': 18, 'tkny_dec': 18, 'tknx_addr': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'tkny_addr': '0x48Fb253446873234F2fEBbF9BdeAA72d9d387f94', 'blocklud': 17342097, 'y': 50079.3888655038, 'yint': 50079.3888655038, 'A': 0.043210678554913784, 'B': 1.0198039027185501, 'pa': 1.129999999999998, 'pb': 1.039999999999986})

In [13]:
c.cid

'4423670769972200025023869896612986749015-0'

In [14]:
seven_days_from_now = int(now.timestamp())+60*60*24*7
seven_days_from_now

1685694661

In [15]:
exclusions0 = {
    '1701411834604692317316873037158841057386-1': 1685428434, # very wide USDC-ETH curve; 23/May
    '4423670769972200025023869896612986749015-0': 1685082834, # vBNT
}

In [16]:
exclusions = {cid for cid, ts in exclusions0.items() if now.timestamp() < ts}

In [17]:
CCc1 = CCc1_noexcl.bycids(exclude=exclusions)
len(CCc1_noexcl), len(CCc1)

(100, 99)

In [18]:
print("\n\n"+"="*100)
print("REMOVED CURVES")
print("="*100)
for cid_ in exclusions:
    print(f"{cid_} [for {(exclusions0[cid_]-now.timestamp())/(60*60*24):3.1f} days more]")



REMOVED CURVES
1701411834604692317316873037158841057386-1 [for 3.9 days more]


#### Create analyzer and pairs

In [19]:
CAc1 = CPCAnalyzer(CCc1)
pairs  = CAc1.pairsc()

### Read prices and create proxy curves

#### Preparations

In [20]:
tokens0 = CAc1.tokens()
tokens0

{'0x0-1AD5',
 'ARB-4ad1',
 'BNT-FF1C',
 'CRETH2-dB64',
 'CRV-cd52',
 'CVX-9D2B',
 'DAI-1d0F',
 'DEXT-C75a',
 'ETH2x_FLI-65BD',
 'HEX-eb39',
 'LBR-aCcA',
 'LHINU-038d',
 'LINK-86CA',
 'LYXe-be6D',
 'MATIC-eBB0',
 'PAXG-Af78',
 'PEPE-1933',
 'RPL-A51f',
 'SMT-7173',
 'TSUKA-69eD',
 'USDC-eB48',
 'USDT-1ec7',
 'WBTC-C599',
 'WETH-6Cc2',
 'XCHF-fc08',
 'rETH-6393',
 'stETH-fE84',
 'vBNT-7f94'}

In [21]:
print("\n\n"+"="*100)
print("REMOVED TOKENS")
print("="*100)



REMOVED TOKENS


In [22]:
REMOVED_TOKENS = {"0x0-1AD5", "LBR-aCcA"}
print(REMOVED_TOKENS)

{'0x0-1AD5', 'LBR-aCcA'}


In [23]:
tokens = tokens0 - REMOVED_TOKENS
pairs  = CAc1.CC.filter_pairs(bothin=tokens)
tokens_addr = {tkn: TA.a(tkn) for tkn in tokens}
tokens_addrr = {v.lower():k for k,v in tokens_addr.items()}
print("\n\n"+"="*100)
print("TOKEN ADDRESSES")
print("="*100)
for k,v in tokens_addr.items():
    print(f"{k:20} {v}")
tokens_addr, tokens_addrr



TOKEN ADDRESSES
DAI-1d0F             0x6B175474E89094C44Da98b954EedeAC495271d0F
CRETH2-dB64          0x49D72e3973900A195A155a46441F0C08179FdB64
WBTC-C599            0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599
SMT-7173             0xB17548c7B510427baAc4e267BEa62e800b247173
HEX-eb39             0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39
LHINU-038d           0xCeDefE438860D2789dA6419b3a19cEcE2A41038d
XCHF-fc08            0xB4272071eCAdd69d933AdcD19cA99fe80664fc08
LINK-86CA            0x514910771AF9Ca656af840dff83E8264EcF986CA
TSUKA-69eD           0xc5fB36dd2fb59d3B98dEfF88425a3F425Ee469eD
PEPE-1933            0x6982508145454Ce325dDbE47a25d4ec3d2311933
RPL-A51f             0xD33526068D116cE69F19A9ee46F0bd304F21A51f
BNT-FF1C             0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C
MATIC-eBB0           0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0
ARB-4ad1             0xB50721BCf8d664c30412Cfbc6cf7a15145234ad1
CRV-cd52             0xD533a949740bb3306d119CC777fa900bA034cd52
CVX-9D2B             0

({'DAI-1d0F': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
  'CRETH2-dB64': '0x49D72e3973900A195A155a46441F0C08179FdB64',
  'WBTC-C599': '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
  'SMT-7173': '0xB17548c7B510427baAc4e267BEa62e800b247173',
  'HEX-eb39': '0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39',
  'LHINU-038d': '0xCeDefE438860D2789dA6419b3a19cEcE2A41038d',
  'XCHF-fc08': '0xB4272071eCAdd69d933AdcD19cA99fe80664fc08',
  'LINK-86CA': '0x514910771AF9Ca656af840dff83E8264EcF986CA',
  'TSUKA-69eD': '0xc5fB36dd2fb59d3B98dEfF88425a3F425Ee469eD',
  'PEPE-1933': '0x6982508145454Ce325dDbE47a25d4ec3d2311933',
  'RPL-A51f': '0xD33526068D116cE69F19A9ee46F0bd304F21A51f',
  'BNT-FF1C': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C',
  'MATIC-eBB0': '0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0',
  'ARB-4ad1': '0xB50721BCf8d664c30412Cfbc6cf7a15145234ad1',
  'CRV-cd52': '0xD533a949740bb3306d119CC777fa900bA034cd52',
  'CVX-9D2B': '0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B',
  'WETH-6Cc2': '0xC02aaA39b

#### CryptoCompare

In [24]:
tokens_cc = [Pair.n(x) for x in tokens]
tokens_cc

['DAI',
 'CRETH2',
 'WBTC',
 'SMT',
 'HEX',
 'LHINU',
 'XCHF',
 'LINK',
 'TSUKA',
 'PEPE',
 'RPL',
 'BNT',
 'MATIC',
 'ARB',
 'CRV',
 'CVX',
 'WETH',
 'USDC',
 'stETH',
 'USDT',
 'ETH2x_FLI',
 'DEXT',
 'rETH',
 'vBNT',
 'LYXe',
 'PAXG']

In [25]:
token_prices_usd_cc = CryptoCompare(apikey=True, verbose=False).query_tokens(tokens_cc)
token_prices_usd_cc

{'DAI': 0.9999,
 'WBTC': 26573.41,
 'SMT': 0.0009693,
 'HEX': 0.01632,
 'XCHF': 1.005,
 'LINK': 6.307,
 'TSUKA': 0.0495,
 'PEPE': 1.45e-06,
 'RPL': 46.97,
 'BNT': 0.4093,
 'MATIC': 0.8986,
 'ARB': 1.145,
 'CRV': 0.8389,
 'CVX': 4.454,
 'WETH': 1811.54,
 'USDC': 0.9999,
 'STETH': 1816.54,
 'USDT': 1.0,
 'DEXT': 0.5233,
 'RETH': 9e-06,
 'LYXE': 10.73,
 'PAXG': 1961.47}

In [26]:
missing_cc = set(tokens_cc) - set(token_prices_usd_cc)
missing_cc

{'CRETH2', 'ETH2x_FLI', 'LHINU', 'LYXe', 'rETH', 'stETH', 'vBNT'}

In [27]:
token_prices_usd = token_prices_usd_cc
P0 = lambda tknb,tknq: token_prices_usd[tknb.upper()]/token_prices_usd[tknq.upper()]
def P(pair):
    try: 
        return P0(*Pair.n(pair).split("/"))
    except KeyError:
        return None

prices_by_pair = {pair: P(pair) for pair in pairs}
prices_n_by_pair = {Pair.n(pair): p for pair, p in prices_by_pair.items()}
print("\n\n"+"="*100)
print("PRICES BY PAIR (CRYPTOCOMPARE)")
print("="*100)
for k,v in prices_n_by_pair.items():
    if not v is None:
        print(f"{k:20} {v:20,.6f}")
    else:
        print(f"{k:20}                  ---")



PRICES BY PAIR (CRYPTOCOMPARE)
DAI/USDT                         0.999900
PEPE/WETH                        0.000000
CRETH2/WETH                           ---
DEXT/USDC                        0.523352
WBTC/USDC                   26,576.067607
SMT/WETH                         0.000001
PAXG/USDC                    1,961.666167
CRV/CVX                          0.188348
LINK/USDC                        6.307631
ARB/MATIC                        1.274204
HEX/WETH                         0.000009
WBTC/USDT                   26,573.410000
DAI/USDC                         1.000000
WBTC/WETH                       14.668961
RPL/XCHF                        46.736318
CRV/USDC                         0.838984
WETH/USDT                    1,811.540000
USDT/USDC                        1.000100
WETH/DAI                     1,811.721172
rETH/WETH                        0.000000
ETH2x_FLI/WETH                        ---
vBNT/BNT                              ---
vBNT/USDC                             ---
W

In [28]:
proxy_curves_cc = [
    CPC.from_pk(p=price, pair=pair, k=1000, cid=cid(pair+"CG"), params=dict(exchange="ccomp")) 
    for pair, price in prices_by_pair.items() if not price is None
]
#proxy_curves_cc

In [29]:
cid

<function __main__.<lambda>(pair)>

#### CoinGecko

In [30]:
addr_s = ",".join(x for x in tokens_addr.values())
url = "https://api.coingecko.com/api/v3/simple/token_price/ethereum"
params = dict(contract_addresses=addr_s, vs_currencies="usd")
r = requests.get(url, params=params)
token_prices_usd_cg_raw = {tokens_addrr[k]: v["usd"] for k,v in r.json().items()}
token_prices_usd_cg = {Pair.n(tokens_addrr[k]).upper(): v["usd"] for k,v in r.json().items()}
token_prices_usd_cg_raw

{'PEPE-1933': 1.46e-06,
 'RPL-A51f': 46.9,
 'DEXT-C75a': 0.524366,
 'HEX-eb39': 0.0163139,
 'USDC-eB48': 0.999862,
 'rETH-6393': 1946.85,
 'USDT-1ec7': 1.0,
 'MATIC-eBB0': 0.899172,
 'BNT-FF1C': 0.410397,
 'ARB-4ad1': 1.14,
 'WBTC-C599': 26552,
 'CRETH2-dB64': 0.324747,
 'LYXe-be6D': 10.91,
 'LHINU-038d': 0.00011033,
 'SMT-7173': 0.059253,
 'XCHF-fc08': 1.11,
 'LINK-86CA': 6.31,
 'CRV-cd52': 0.83842,
 'DAI-1d0F': 0.999884,
 'stETH-fE84': 1813.76,
 'WETH-6Cc2': 1814.75,
 'vBNT-7f94': 0.304342,
 'ETH2x_FLI-65BD': 11.68,
 'PAXG-Af78': 1971.63,
 'CVX-9D2B': 4.45,
 'TSUKA-69eD': 0.04953971}

In [31]:
missing_cg = set(tokens_addr) - set(token_prices_usd_cg_raw)
missing_cg

set()

In [32]:
token_prices_usd = token_prices_usd_cg
P0 = lambda tknb,tknq: token_prices_usd[tknb.upper()]/token_prices_usd[tknq.upper()]
def P(pair):
    try: 
        return P0(*Pair.n(pair).split("/"))
    except KeyError:
        return None

prices_by_pair = {pair: P(pair) for pair in pairs}
prices_n_by_pair = {Pair.n(pair): p for pair, p in prices_by_pair.items()}
print("\n\n"+"="*100)
print("PRICES BY PAIR (COINGECKO)")
print("="*100)
for k,v in prices_n_by_pair.items():
    if not v is None:
        print(f"{k:20} {v:20,.6f}")
    else:
        print(f"{k:20}                  ---")



PRICES BY PAIR (COINGECKO)
DAI/USDT                         0.999884
PEPE/WETH                        0.000000
CRETH2/WETH                      0.000179
DEXT/USDC                        0.524438
WBTC/USDC                   26,555.664682
SMT/WETH                         0.000033
PAXG/USDC                    1,971.902122
CRV/CVX                          0.188409
LINK/USDC                        6.310871
ARB/MATIC                        1.267833
HEX/WETH                         0.000009
WBTC/USDT                   26,552.000000
DAI/USDC                         1.000022
WBTC/WETH                       14.631216
RPL/XCHF                        42.252252
CRV/USDC                         0.838536
WETH/USDT                    1,814.750000
USDT/USDC                        1.000138
WETH/DAI                     1,814.960535
rETH/WETH                        1.072792
ETH2x_FLI/WETH                   0.006436
vBNT/BNT                         0.741579
vBNT/USDC                        0.304384
WETH/

In [33]:
proxy_curves_cg = [
    CPC.from_pk(p=price, pair=pair, k=1000, cid=cid(pair+"CG"), params=dict(exchange="cgecko")) 
    for pair, price in prices_by_pair.items() if not price is None
]
#proxy_curves_cg

#### Assembly

In [34]:
# CCother = CCu3.bypairs(CCc1.pairs())
CCcg = CPCContainer(proxy_curves_cg)
CCcc = CPCContainer(proxy_curves_cc)
CCfull = CCc1.copy().add(CCcg).add(CCcc)
#CAother = CPCAnalyzer(CCother)
CAfull = CPCAnalyzer(CCfull)
CAnc1  = CPCAnalyzer(CCnc1)
print(f"CAfull: {len(CAfull.CC):4} entries")
print(f"CAnc1:  {len(CAnc1.CC):4} entries")

CAfull:  158 entries
CAnc1:    42 entries


## By-pair data for Carbon

### Count by pairs

In [35]:
dfc1 = CAc1.count_by_pairs().rename(columns=dict(count="carbon")).astype(str)
dfnc1 = CAnc1.count_by_pairs().rename(columns=dict(count="other")).astype(str)
print("\n\n"+"="*100)
print("AVAILABLE PAIRS (CARBON AND OTHER)")
print("="*100)
df = pd.concat([dfc1, dfnc1], axis=1).fillna("").sort_index()
print(df)
pairs_df = df
#df



AVAILABLE PAIRS (CARBON AND OTHER)
                         carbon other
pair                                 
0x0-1AD5/WETH-6Cc2            2      
ARB-4ad1/MATIC-eBB0           2      
ARB-4ad1/WETH-6Cc2                  1
BNT-FF1C/USDC-eB48            3     1
CRETH2-dB64/WETH-6Cc2         2     1
CRV-cd52/CVX-9D2B             2      
CRV-cd52/USDC-eB48            2     1
CRV-cd52/WETH-6Cc2                  1
CVX-9D2B/USDC-eB48                  1
CVX-9D2B/WETH-6Cc2                  1
DAI-1d0F/USDC-eB48            3     2
DAI-1d0F/USDT-1ec7            2     2
DEXT-C75a/USDC-eB48           2      
DEXT-C75a/WETH-6Cc2                 1
ETH2x_FLI-65BD/WETH-6Cc2      1     1
HEX-eb39/USDC-eB48                  1
HEX-eb39/WETH-6Cc2            2      
LBR-aCcA/WETH-6Cc2            1      
LHINU-038d/USDT-1ec7          1      
LHINU-038d/WETH-6Cc2                1
LINK-86CA/USDC-eB48           1     1
LINK-86CA/USDT-1ec7           2     1
LYXe-be6D/USDC-eB48           1     1
MATIC-eBB0/WE

In [36]:
dfc1 = CAc1.count_by_pairs().rename(columns=dict(count="carbon")).astype(str)
dfnc1 = CAnc1.count_by_pairs().rename(columns=dict(count="other")).astype(str)
print("\n\n"+"="*100)
print("CARBON PAIRS NOT MATCHED")
print("="*100)
print(df[df["other"]==""])



CARBON PAIRS NOT MATCHED
                     carbon other
pair                             
0x0-1AD5/WETH-6Cc2        2      
ARB-4ad1/MATIC-eBB0       2      
CRV-cd52/CVX-9D2B         2      
DEXT-C75a/USDC-eB48       2      
HEX-eb39/WETH-6Cc2        2      
LBR-aCcA/WETH-6Cc2        1      
LHINU-038d/USDT-1ec7      1      
RPL-A51f/XCHF-fc08        1      
rETH-6393/WBTC-C599       1      
vBNT-7f94/USDC-eB48       1      


In [37]:
dfc1 = CAc1.count_by_pairs().rename(columns=dict(count="carbon")).astype(str)
dfnc1 = CAnc1.count_by_pairs().rename(columns=dict(count="other")).astype(str)
print("\n\n"+"="*100)
print("OTHER PAIRS WITH NO CARBON")
print("="*100)
print(df[df["carbon"]==""])



OTHER PAIRS WITH NO CARBON
                     carbon other
pair                             
ARB-4ad1/WETH-6Cc2              1
CRV-cd52/WETH-6Cc2              1
CVX-9D2B/USDC-eB48              1
CVX-9D2B/WETH-6Cc2              1
DEXT-C75a/WETH-6Cc2             1
HEX-eb39/USDC-eB48              1
LHINU-038d/WETH-6Cc2            1
MATIC-eBB0/WETH-6Cc2            1
PEPE-1933/USDC-eB48             1
RPL-A51f/WETH-6Cc2              1
Silo-B1f8/USDC-eB48             1
XCHF-fc08/WETH-6Cc2             1
eRSDL-D3A6/WETH-6Cc2            1


In [38]:
print("\n\n                    CARBON     CGECKO   CCOMP")
print(f"Pairs:                {len(pairs):4}    {len(CCcg.pairs()):7} {len(CCcc.pairs()):7}")
print(f"Tokens:               {len(tokens):4}    {len(CCcg.tokens()):7} {len(CCcc.tokens()):7}")
print(f"Curves:               {len(CAc1.CC):4}    {len(CCcg):7} {len(CCcc):7}")



                    CARBON     CGECKO   CCOMP
Pairs:                  32         32      27
Tokens:                 26         26      22
Curves:                 99         32      27


### Calculate by-pair statistics

In [39]:
print("\n\n")
print("*"*100)
print(f"BY-PAIR DATA")
print("*"*100)




****************************************************************************************************
BY-PAIR DATA
****************************************************************************************************


In [40]:
pasdf    = CAfull.pool_arbitrage_statistics()
pasnc1df = CAnc1.pool_arbitrage_statistics(only_pairs_with_carbon=False)

### Print by-pair statistics

In [41]:
def prints(*x):
    global s
    s += " ".join([str(x_) for x_ in x])
    s += "\n"
out_by_pair = dict()
carbon_by_pair = dict()
other_by_pair = dict()

for pair in list(pairs):
    s = ""
    prints("\n\n"+"="*100)
    prints(f"Pair = {pair}")
    prints("="*100)
    df = pasdf.loc[Pair.n(pair)]
    try:
        nc1df = pasnc1df.loc[Pair.n(pair)]
    except:
        nc1df = pd.DataFrame()
    hasproxydata = len(df.reset_index()[df.reset_index()["exchange"]=="cgecko"])>0
    if hasproxydata:
        prints("\n--- ALL CARBON AND REFERENCE POSITIONS ---")
        prints(df.to_string())
        carbon_by_pair[pair] = [[k,v] for k,v in df.to_dict(orient="index").items()]
        prints("\n--- IN-THE-MONEY POSITIONS ---")
        dfitm = df[df["itm"]=="x"]
        if len(dfitm) > 0:
            prints(dfitm.to_string())
        else:
            prints("-None-")
        prints("\n--- ALL NON-CARBON POSITIONS ---")
        if len(nc1df) > 0:
            prints(nc1df.to_string())
        else:
            prints("-None-")
        other_by_pair[pair] = [[k,v] for k,v in nc1df.to_dict(orient="index").items()]
        
    else:
        prints("\n--- NO PRICE DATA AVAILABLE ---")
    
    out_by_pair[pair] = s
    print(s)




Pair = DAI-1d0F/USDT-1ec7

--- ALL CARBON AND REFERENCE POSITIONS ---
                       price         vl itm  b  s                               bsv
exchange  cid0                                                                     
carbon_v1 268742-0  0.995000  20.100501      b          buy-DAI @ 1.00 USDT per DAI
cgecko    e50e04c1  0.999884  63.249222      b  s  buy-sell-DAI @ 1.00 USDT per DAI
ccomp     e50e04c1  0.999900  63.248716      b  s  buy-sell-DAI @ 1.00 USDT per DAI
carbon_v1 268742-1  1.005000  20.000000         s      sell-DAI @ 1.00 USDT per DAI

--- IN-THE-MONEY POSITIONS ---
-None-

--- ALL NON-CARBON POSITIONS ---
                        price             vl itm  b  s                               bsv
exchange   cid0                                                                         
uniswap_v3 d32c192e  0.999891  160200.932541      b  s  buy-sell-DAI @ 1.00 USDT per DAI
           6e83e219  0.999978   64932.025074      b  s  buy-sell-DAI @ 1.00 USDT per

## Summary data

In [42]:
print("\n\n")
print("*"*100)
print(f"SUMMARY DATA")
print("*"*100)




****************************************************************************************************
SUMMARY DATA
****************************************************************************************************


### Create summary data

In [43]:
itmcarbdf = pasdf.query("exchange == 'carbon_v1'").query("itm == 'x'")

In [44]:
itmcarb_pairs = sorted({x[0] for x in tuple(itmcarbdf.index)})
itmcarb_pairs

['ARB/MATIC',
 'CRETH2/WETH',
 'CRV/USDC',
 'DEXT/USDC',
 'HEX/WETH',
 'LINK/USDT',
 'WBTC/USDC',
 'WBTC/WETH',
 'WETH/BNT',
 'WETH/USDT',
 'rETH/WBTC',
 'rETH/WETH',
 'vBNT/BNT']

In [45]:
itmcarb_pos = itmcarbdf.reset_index().to_dict(orient="records")
itmcarb_pos[:2]

[{'pair': 'ARB/MATIC',
  'exchange': 'carbon_v1',
  'cid0': '806240-1',
  'price': 1.4285714285714268,
  'vl': 141.80602335295742,
  'itm': 'x',
  'b': 'b',
  's': '',
  'bsv': 'buy-ARB @ 1.43 MATIC per ARB'},
 {'pair': 'CRETH2/WETH',
  'exchange': 'carbon_v1',
  'cid0': '092712-1',
  'price': 0.990099009900981,
  'vl': 6.029912872749975,
  'itm': 'x',
  'b': 'b',
  's': '',
  'bsv': 'buy-CRETH2 @ 0.99 WETH per CRETH2'}]

In [46]:
itmcarb_pos_bypair = {
    pair: [x for x in itmcarb_pos if x["pair"] == pair]
    for pair in itmcarb_pairs
}
#itmcarb_pos_bypair

In [47]:
missing_pairs = [pair for pair, price in prices_by_pair.items() if price is None]
missing_pairs

[]

### Convert summary data to Telegram

In [48]:
telegram_data = dict(
    script_version = __SCRIPT_VERSION__,         # version number of the script producing this record
    script_version_dt = __SCRIPT_DATE__,         # ditto date
    time_ts = int(now.timestamp()),              # timestamp (epoch)
    time_iso = now.isoformat().split('.')[0],    # timestap (iso format)
    prices_usd = token_prices_usd,               # token prices (usd)
    pairs0 = pairs_df.to_dict(orient="index"),   # carbon pairs and other pairs count       
    pairs = list(pairs),                         # all carbon pairs
    pairs_n = len(pairs),                        # ...number
    curves_n = len(CCc1),                        # number of Carbon curves
    itm_pairs = itmcarb_pairs,                   # pairs that have curves in the money (list)
    itm_pairs_n = len(itmcarb_pairs),            # ...number
    itm_pos = itmcarb_pos,                       # carbon and reference positions that are in the money (list)
    itm_pos_n = len(itmcarb_pos),                # ...number
    all_pos_bp = carbon_by_pair,                 # all carbon and reference positions by pair (dict->list)
    all_pos_bp_n = len(carbon_by_pair),          # ...number
    other_pos_bp = other_by_pair,                # all other positions (dict->list)
    other_pos_bp_n = len(other_by_pair),         # ...number
    itm_pos_bypair = itmcarb_pos_bypair,         # ditto, but dict[pair] -> list
    missing_pairs = missing_pairs,               # missing pairs
    missing_pairs_n = len(missing_pairs),        # ...number
    removed_curves = list(exclusions),           # curves that have been explicitly removed
    removed_curves_n = len(exclusions),          # ...number
    removed_tokens = list(REMOVED_TOKENS),       # removed tokens
    removed_tokens_n = len(REMOVED_TOKENS),      # ...number
    out_by_pair = out_by_pair                    # output by pair
)

In [49]:
td = telegram_data
summary_data = dict()
s = ""
s += f"="*47
s += f"\nARBITRAGE RUN @ {td['time_iso']}Z\n"
s += f"="*47+"\n"
s += f"Removed tokens:          {td['removed_tokens_n']:3}\n"
s += f"Total pairs:             {td['pairs_n']:3}\n"
s += f"Missing pairs:           {td['missing_pairs_n']:3}\n"
s += f"Removed curves:          {td['removed_curves_n']:3}\n"
s += f"In-the-money pairs:      {td['itm_pairs_n']:3}\n"
s += f"Total curves:            {td['curves_n']:3}\n"
s += f"In-the-money curves:     {td['itm_pos_n']:3}\n"
total_vl_usd = 0
total_arbval = 0
s += "-----------------------------------------------\n"
s += "PAIR         CID          VLOCK    ARBPC    VAL\n"
s += "-----------------------------------------------\n"
for p in td['itm_pos']:
    price_pair = prices_n_by_pair[p['pair']] or 0
    price_pc0 = abs(price_pair/p['price']-1)
    price_pc = f"{price_pc0*100:8.1f}%"
    vl_token = p['pair'].split('/')[0].split("-")[0]
    vl_token_price = token_prices_usd.get(vl_token.upper())
    vl_usd = p['vl']*vl_token_price
    total_vl_usd += vl_usd
    arbval = vl_usd * abs(price_pair/p['price']-1)
    if price_pc.endswith("100.0%"): 
        price_pc = "         "
        arbval = 0
    total_arbval += arbval
    d = dict(
        pair = p['pair'],
        cid0 = p["cid0"],
        vl_usd = vl_usd,
        price_pc = price_pc0,
        arbval = arbval,
        price = price_pair,
    )
    summary_data[p["cid0"]] = d
    #print(d)
    s += f"{p['pair']:12} "
    s += f"{p['cid0'][-8:]:8} "
    s += f"{vl_usd:9,.0f}"
    s += f"{price_pc} "
    s += f"{arbval:6,.0f}"
    #s += f"[{p['bsv']}; p={price_pair:,.2f}]"
    #s += f"\n{p}"
    s += "\n"
s += "-----------------------------------------------\n"
s += f"TOTAL {total_vl_usd:25,.0f}   {100*total_arbval/total_vl_usd:5.1f}% {total_arbval:6,.0F}\n"
s += "===============================================\n"
s += """
All numbers in USDC. Figures above are upper
bounds, not estimates. False positives are to
be expected, but not false negatives.\n
"""[1:]

telegram_data["summary_text"] = s
telegram_data["summary_data"] = summary_data
print()
print(s)


ARBITRAGE RUN @ 2023-05-26T09:31:01Z
Removed tokens:            2
Total pairs:              32
Missing pairs:             0
Removed curves:            1
In-the-money pairs:       13
Total curves:             99
In-the-money curves:      19
-----------------------------------------------
PAIR         CID          VLOCK    ARBPC    VAL
-----------------------------------------------
ARB/MATIC    806240-1       162    11.3%     18
CRETH2/WETH  092712-1         2               0
CRV/USDC     612490-0         1     6.8%      0
DEXT/USDC    669784-0         1    12.6%      0
HEX/WETH     881242-0         3    64.0%      2
LINK/USDT    960408-0         0     8.6%      0
WBTC/USDC    537493-0       482     1.9%      9
WBTC/WETH    709391-0     1,076     1.1%     12
WETH/BNT     326076-0         5     6.2%      0
WETH/USDT    691656-0         5     4.0%      0
rETH/WBTC    727077-0         2     2.7%      0
rETH/WETH    903202-1       510     0.8%      4
rETH/WETH    903202-0       486     0.2

## Notifications

In [50]:
# data frame with arbitrage opportunities...
arbdf = pd.DataFrame.from_dict(summary_data.values())[["cid0", "arbval"]].set_index("cid0")
#arbdf

In [51]:
# ...and data frame of all tracked positions, arb or not...
ciddf = pd.DataFrame([[c.cid0 for c in CCc1],[0]*len(CCc1)], index="cid0,arbval".split(",")).T.set_index("cid0")
#ciddf

In [52]:
# ...combined into one dataframe (arb first)
notifdf = arbdf.combine_first(ciddf)
#notifdf

In [53]:
# read the dataframe of previous arb notification levels...
try:
    notifcdf0 = pd.read_csv("Analysis_015.notifdf.csv").set_index("cid0")
except:
    print("Creating new nofifdf")
    notifcdf0 = ciddf
notifcdf = notifcdf0.copy()

In [54]:
notifcdf0.loc["749015-0"]

arbval     2457.018289
arbval1    2457.018289
Name: 749015-0, dtype: float64

In [55]:
# ...and augment it with current arb levels
notifcdf["arbval1"] = notifdf["arbval"]
notifcdf.loc["749015-0"]

arbval     2457.018289
arbval1    2469.315562
Name: 749015-0, dtype: object

In [56]:
# notification is due where level goes from < 50 to > 50
notifbreachdf = notifcdf.query("arbval1>=50 and arbval<50")
notifbreachdf = notifbreachdf.drop("arbval", axis=1).rename(columns={"arbval1": "arbval"})
notifbreachdf

,arbval
cid0,


In [57]:
# update the previous notifications df with the current notifications
notifndf = notifbreachdf.combine_first(notifcdf0)
notifndf.to_csv("Analysis_015.notifdf.csv")

In [58]:
notifndf.loc["749015-0"]

arbval     2457.018289
arbval1    2457.018289
Name: 749015-0, dtype: float64

In [59]:
# create all new notifications
notif_str = "".join([
    "[{td[time_iso]}::{td[time_ts]}] |new| == {d}\n".format(
        cid0=cid0, 
        td=td, 
        d = json.dumps(dict(
            pair = summary_data[cid0]["pair"],
            cid0=cid0,
            arbval = x["arbval"],
            vl_usd = summary_data[cid0]["vl_usd"],
            price = summary_data[cid0]["price"],
            #sd = summary_data[cid0],
        ))
    )
    for cid0, x in notifbreachdf.to_dict(orient="index").items()
])
notif_str

''

In [60]:
# print notifications (if any)
if notif_str:
    s0 = f"="*47
    s0 += f"\nNOTICATIONS\n"
    s0 += f"="*47
    print(s0)
    print(notif_str)

## Save some files

In [61]:
with open("Analysis_015.notifications", "a") as f:
    f.write(notif_str)

In [62]:
with open("Analysis_015.latest.out", "w") as f:
    f.write(telegram_data["summary_text"])

In [63]:
with open("Analysis_015.latest.json", "w") as f:
    f.write(json.dumps(telegram_data))

## Review

In [64]:
#print(CCfull.bycids(endswith="612490-0")[0].description())